Modify this line to briefly discribe the functionality of pubmedpy.ipynb<br/><br/>Copyright (C) 2017  Martin Engqvist Lab<br/>This program is free software: you can redistribute it and/or modify<br/>it under the terms of the GNU General Public License as published by<br/>the Free Software Foundation, either version 3 of the License, or<br/>(at your option) any later version.<br/>This program is distributed in the hope that it will be useful,<br/>but WITHOUT ANY WARRANTY; without even the implied warranty of<br/>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the<br/>GNU General Public License for more details.<br/>You should have received a copy of the GNU General Public License<br/>along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [7]:
import os
from dotenv import load_dotenv, find_dotenv
from os.path import join, dirname, basename, exists, isdir

### Load environmental variables from the project root directory ###
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

# now you can get the variables using their names

# Check whether a network drive has been specified
DATABASE = os.environ.get("NETWORK_URL")
if DATABASE == 'None':
    pass
else:
    pass
    #mount network drive here

# set up directory paths
CURRENT_DIR = os.getcwd()
PROJ = dirname(dotenv_path) # project root directory

DATA = join(PROJ, 'data') #data directory
RAW_EXTERNAL = join(DATA, 'raw_external') # external data raw directory
RAW_INTERNAL = join(DATA, 'raw_internal') # internal data raw directory
INTERMEDIATE = join(DATA, 'intermediate') # intermediate data directory
FINAL = join(DATA, 'final') # final data directory

RESULTS = join(PROJ, 'results') # output directory
FIGURES = join(RESULTS, 'figures') # figure output directory
PICTURES = join(RESULTS, 'pictures') # picture output directory


# make folders specific for certain data
folder_name = ''
if folder_name != '':
    #make folders if they don't exist
    if not exists(join(RAW_EXTERNAL, folder_name)):
        os.makedirs(join(RAW_EXTERNAL, folder_name))

    if not exists(join(INTERMEDIATE, folder_name)):
        os.makedirs(join(INTERMEDIATE, folder_name))

    if not exists(join(FINAL, folder_name)):
        os.makedirs(join(FINAL, folder_name))

print('Standard variables loaded, you are good to go!')

Standard variables loaded, you are good to go!


In [170]:
import json
from pubchempy import Compound
from chemspipy import ChemSpider
cs = ChemSpider('0yoSpsZCYrwWqNfZXOsclxqX09Xhzt3v')

#from pubchempy import Compound , get_compounds

from rdkit import Chem
from rdkit.Chem import Descriptors
filepath = join(RAW_EXTERNAL,"BRENDA_data_2019_1","2019-04-02_substrate_cache.json")
with open(filepath, 'r') as f:
        data = json.loads(f.read())
        
def separate_nones(dicti):
    Nones = []
    Smiles = {}
    for key in dicti:
        if dicti[key] == None:
            Nones.append(key)
        else:
            Smiles[key] = dicti[key]
    return Smiles, Nones

def filter_irrelevant(listi):
    unwanted_terms = ['protein', 'ase', 'factor', 'rna', 'dna', 'subunit', 'ribosom', 'receptor', 'active', 'cell']
    Uninteresting = [x for x in listi if any(word in x for word in unwanted_terms)]
    Still_interesting = []
    print("Filtering out uninteresting compounds.")
    for i in range(0,len(listi)):
        if listi[i] not in Uninteresting:
            Still_interesting.append(listi[i])
    print("Filtered out " + str(len(Uninteresting)) + " compounds. " + str(len(Still_interesting)) + " compounds of interest remain.")
    return Still_interesting,Uninteresting       


def test_smiles(dicti,listi):
    """Takes as input a dictionary of substrate names and their SMILES, converts to mol-format and returns a dictionary of substrate names and 
    and their respective molecular weight."""
    Smiles = {}
    Nones = listi
    for key in dicti:
        try:
            mol = Chem.MolFromSmiles(dicti[key])
            Smiles[key] = dicti[key]
        except:
            print("The shit hit the fan!")
            Nones.append(key)
    print(str(len(Smiles)) +" SMILES passed." + str(len(Nones)) + " unidentified substrates remain")    
    return Smiles, Nones

def smiles_from_pubchem(listi):
    NoSmiles = []
    NewSmiles = {}
    for i in range(0,len(listi)):
        x = get_compounds(listi[i],'name')
        if len(x) > 0:
            for result in x:            
                NewSmiles[listi[i]] = result.canonical_smiles
                print('Added ' + listi[i] + ' [' + NewSmiles[listi[i]] +"]")
        else:
            NoSmiles.append(listi[i])
    print("Found " + str(len(NewSmiles)) + " new SMILES. Returning " + str(len(NoSmiles)) + " undetermined substrates.")
    return NewSmiles, NoSmiles

def smiles_from_chemspider(listi):
    Nones = []
    for i in range(0,len(listi)):
        print('Now trying ' + str(listi[i]))
        for results in cs.search(listi[i]):
            print(results.smiles)            


Nones = []
Smiles = {}
for key in data:
    if data[key] == None:
        Nones.append(key)
    else:
        Smiles[key] = data[key]



Nones2 = smiles_from_pubchem(Nones)

Added cobalt-sirohydrochlorin [CC1(C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(C(C(=N5)C=C1[N-]2)CCC(=O)O)(C)CC(=O)O)CC(=O)O)CCC(=O)O)C(=C3CC(=O)O)CCC(=O)O)CCC(=O)O)CC(=O)O.[Co]]
Added cobalt-sirohydrochlorin [CC1(C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(C(C(=N5)C=C1[N-]2)CCC(=O)O)(C)CC(=O)O)CC(=O)O)CCC(=O)O)C(=C3CC(=O)O)CCC(=O)O)CCC(=O)O)CC(=O)O.[Co+2]]
Added cobalt-sirohydrochlorin [CC1(C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(C(C(=N5)C=C1[N-]2)CCC(=O)[O-])(C)CC(=O)[O-])CC(=O)[O-])CCC(=O)[O-])C(=C3CC(=O)[O-])CCC(=O)[O-])CCC(=O)[O-])CC(=O)[O-].[Co]]
Added 6,6-kestotetraose [C(C1C(C(C(C(O1)OC2(C(C(C(O2)COC3(C(C(C(O3)COC4(C(C(C(O4)CO)O)O)CO)O)O)CO)O)O)CO)O)O)O)O]
Added (e)-1-(glutathion-s-yl)-n-hydroxy-2-(1h-indol-3-yl)ethan-1-imine [C1=CC=C2C(=C1)C(=CN2)CC(=NO)SCC(C(=O)NCC(=O)[O-])NC(=O)CCC(C(=O)[O-])[NH3+]]
Added guanosine 3'-diphosphate 5'-diphosphate [C1=NC2=C(N1C3C(C(C(O3)COP(=O)(O)OP(=O)(O)O)OP(=O)(O)OP(=O)(O)O)O)N=C(NC2=O)N]
Added galactocerebroside [CCCCCCCCCCCCCC=CC(C(CO)NC(=O)OC1C(C(C(C(O1)CO)O

Added aprotinin [CCC(C)C1C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NCC(=O)NC(C(=O)NC2CSSCC3C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(CSSCC(C(=O)NC(C(=O)NC(C(=O)N4CCCC4C(=O)N5CCCC5C(=O)NC(C(=O)NC(C(=O)NCC(=O)N6CCCC6C(=O)NC(CSSCC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N3)CC(=O)O)CCC(=O)O)C)CO)CCCCN)CC7=CC=CC=C7)CC(=O)N)CC(=O)N)CCCNC(=N)N)CCCCN)C)CCCNC(=N)N)NC(=O)CNC(=O)CNC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC2=O)CCC(=O)N)C(C)O)CC8=CC=CC=C8)C(C)C)CC9=CC=C(C=C9)O)C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N1)CCCNC(=N)N)C)CCCCN)C(C)O)CC1=CC=C(C=C1)O)CCC(=O)O)CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C(CC(=O)O)NC(=O)C1CCCN1C(=O)C(CCCNC(=N)N)N)C(=O)NCC(=O)NCC(=O)NC(C)C(=O)O)C(C)O)CCCNC(=N)N)CCSC)CC(C)C)C)CCCCN)C)CC(=O)N)CC1=CC=C(C=C1)O)CC1=CC=CC=C1)CC1=CC=C(C=C1)O)CCCNC(=N)N)C(C)CC]
Added aprotinin [CCC(C)C1C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NCC(=O)NC(C(=O)NC2CSSCC3C(=O)NC(C(=O)NC(C(=

Added bacteriochlorophyll a [CCC1C(C2=NC1=CC3=C(C4=C([N-]3)C(=C5C(C(C(=N5)C=C6C(=C(C(=C2)[N-]6)C(=O)C)C)C)CCC(=O)OCC=C(C)CCCC(C)CCCC(C)CCCC(C)C)C(C4=O)C(=O)OC)C)C.[Mg+2]]
Added bacteriochlorophyll a [CCC1C(C2=NC1=CC3=C(C4=C([N-]3)C(=C5C(C(C(=N5)C=C6C(=C(C(=C2)[N-]6)C(=O)C)C)C)CCC(=O)OCC=C(C)CCCC(C)CCCC(C)CCCC(C)C)C(C4=O)C(=O)OC)C)C.[Mg+2]]
Added bacteriochlorophyll a [CCC1C(C2=[NH+]C1=CC3=C(C4=C([N-]3)C(=C5C(C(C(=[NH+]5)C=C6C(=C(C(=C2)[N-]6)C(=O)C)C)C)CCC(=O)OCC=C(C)CCCC(C)CCCC(C)CCCC(C)C)C(C4=O)C(=O)OC)C)C.[Mg+2]]
Added 4'-demethylrebeccamycin [C1=CC2=C(C(=C1)Cl)NC3=C4C(=C5C(=C23)C(=O)NC5=O)C6=C(N4C7C(C(C(C(O7)CO)O)O)O)C(=CC=C6)Cl]
Added 4-amino-4-deoxy-alpha-l-arabinopyranosyl ditrans,octacis-undecaprenyl phosphate [CC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCOP(=O)([O-])OC1C(C(C(CO1)[NH3+])O)O)C)C)C)C)C)C)C)C)C)C)C]
Added 4-amino-4-deoxy-alpha-l-arabinopyranosyl ditrans,octacis-undecaprenyl phosphate [CC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CC

Added (2s)-liquiritigenin [C1C(OC2=C(C1=O)C=CC(=C2)O)C3=CC=C(C=C3)O]
Added myelin basic protein [CC(C)CC(C(=O)O)NC(=O)C(CC1=CC=C(C=C1)O)NC(=O)C(CCCCN)NC(=O)C(CO)NC(=O)C(CCCN=C(N)N)NC(=O)C(CCC(=O)N)NC(=O)C(CO)NC(=O)C2CCCN2C(=O)C(CCCN=C(N)N)NC(=O)C(CCCCN)NC(=O)C(CCC(=O)N)N]
Added gdp-4-dehydro-6-deoxy-d-talose [CC1C(=O)C(C(C(O1)OP(=O)(O)OP(=O)(O)OCC2C(C(C(O2)N3C=NC4=C3N=C(NC4=O)N)O)O)O)O]
Added ferriprotoporphyrin ix [CC1=C(C2=CC3=C(C(=C([N-]3)C=C4C(=C(C(=N4)C=C5C(=C(C(=N5)C=C1[N-]2)C=C)C)C=C)C)C)CCC(=O)O)CCC(=O)O.[Cl-].[Fe+3]]
Added ferriprotoporphyrin ix [CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C(=N5)C=C1[N-]2)C)CCC(=O)[O-])CCC(=O)[O-])C)C(=C3C)C=C)C=C.[Fe]]
Added udp-murnac-l-ala-d-glu-l-lys-d-ala-d-ala [CC(C(=O)NC(CCC(=O)N)C(=O)NC(CCCC[NH3+])C(=O)NC(C)C(=O)NC(C)C(=O)[O-])NC(=O)C(C)OC1C(C(OC(C1O)CO)OP(=O)([O-])OP(=O)([O-])OCC2C(C(C(O2)N3C=CC(=O)NC3=O)O)O)NC(=O)C]
Added z-phe-arg-7-amido-4-methylcoumarin [CC1=CC(=O)OC2=C1C=CC(=C2)NC(=O)C(CCCN=C(N)N)NC(=O)C(CC3=CC=CC=C3)NC(=O)OCC4=CC

Added citrus pectin [C(C(C(C(C=O)O)O)O)O]
Added (1r,6r)-6-hydroxy-2-succinylcyclohexa-2,4-diene-1-carboxylate [C1=CC(C(C(=C1)C(=O)CCC(=O)O)C(=O)O)O]
Added 1-palmitoyl-2-lysophosphatidylcholine [CCCCCCCCCCCCCCCC(=O)OCC(COP(=O)([O-])OCC[N+](C)(C)C)O]
Added casomorphin [CCC(C)C(C(=O)O)NC(=O)C1CCCN1C(=O)C(CCC(=O)O)NC(=O)C(C(C)C)NC(=O)C(CC2=CC=CC=C2)NC(=O)C3CCCN3C(=O)C(CC4=CC=C(C=C4)O)N]
Added enterostatin [CC(C(=O)N1CCCC1C(=O)NCC(=O)N2CCCC2C(=O)NC(CCCN=C(N)N)C(=O)O)N]
Added udp-2,3-diacetamido-2,3-dideoxy-alpha-d-glucuronate [CC(=O)NC1C(C(OC(C1O)C(=O)[O-])OP(=O)([O-])OP(=O)([O-])OCC2C(C(C(O2)N3C=CC(=O)NC3=O)O)O)NC(=O)C]
Added 1-archaetidyl-1d-myo-inositol 3-phosphate [CC(C)CCCC(C)CCCC(C)CCCC(C)CCOCC(COP(=O)(O)OC1C(C(C(C(C1O)OP(=O)(O)O)O)O)O)OCCC(C)CCCC(C)CCCC(C)CCCC(C)C]
Added 3-o-alpha-d-glucopyranosyl-d-glucopyranose [C(C1C(C(C(C(O1)O)O)OC2C(C(C(C(O2)CO)O)O)O)O)O]
Added (2s)-naringenin [C1C(OC2=CC(=CC(=C2C1=O)O)O)C3=CC=C(C=C3)O]
Added n-carbamoyl-l-methionine [CSCCC(C(=O)[O-])NC(=O)N]
Ad

Added nomega-(adp-d-ribosyl)-l-arginine [C1=NC2=C(C(=N1)N)N=CN2C3C(C(C(O3)COP(=O)(O)OP(=O)(O)OCC4C(C(C(O4)NC(=NCCCC(C(=O)O)N)N)O)O)O)O]
Added 2-epi-5-epi-valiolone 7-phosphate [C1C(=O)C(C(C(C1(COP(=O)([O-])[O-])O)O)O)O]
Added 2-epi-5-epi-valiolone 7-phosphate [C1C(=O)C(C(C(C1(COP(=O)(O)O)O)O)O)O]
Added n-(4-aminobutyl)-l-glutamine [C(CCNC(=O)CCC(C(=O)O)N)CN]
Added udp-d-galacturonic acid [C1=CN(C(=O)NC1=O)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OC3C(C(C(C(O3)C(=O)O)O)O)O)O)O]
Added tgfbeta [CCC(C)C(C(=O)NC(CCC(=O)N)C(=O)NC(CO)C(=O)NC(C)C(=O)NCC(=O)NC(CC(C)C)C(=O)NC(C(C)O)C(=O)NC(CCC(=O)O)C(=O)NC(C(C)O)C(=O)NC(CC(C)C)C(=O)NC(CC(=O)N)C(=O)NC(CCCNC(=N)N)C(=O)NC(CCC(=O)O)C(=O)NCC(=O)NC(C(C)C)C(=O)NC(CC1=CC=C(C=C1)O)C(=O)NC(C(C)O)C(=O)NC(C(C)C)C(=O)NC(CC2=CC=CC=C2)C(=O)NC(C)C(=O)N3CCCC3C(=O)NC(C(C)O)C(=O)NC(CC(=O)N)C(=O)NC(CCC(=O)O)C(=O)NC(C)C(=O)NC(CC4=CC=CC=C4)C(=O)NC(CCCNC(=N)N)C(=O)NC(C)C(=O)NC(CC(C)C)C(=O)N5CCCC5C(=O)N6CCCC6C(=O)NC(CCCNC(=N)N)C(=O)NC(CCC(=O)O)C(=O)NC(CCCNC(=N)N)C(=O)NC(CO)C(=O)N

Added hemopressin [CC(C)CC(C(=O)NC(CO)C(=O)NC(CC1=CN=CN1)C(=O)O)NC(=O)C(CC2=CC=CC=C2)NC(=O)C(CCCCN)NC(=O)C(CC3=CC=CC=C3)NC(=O)C(CC(=O)N)NC(=O)C(C(C)C)NC(=O)C4CCCN4]
Added coproporphyrinogen-iii [CC1=C2CC3=C(C(=C(N3)CC4=C(C(=C(N4)CC5=C(C(=C(N5)CC(=C1CCC(=O)O)N2)C)CCC(=O)O)C)CCC(=O)O)CCC(=O)O)C]
Added dtdp-3,4-didehydro-2,6-dideoxy-alpha-d-glucose [CC1C(=O)C(=O)CC(O1)OP(=O)(O)OP(=O)(O)OCC2C(CC(O2)N3C=C(C(=O)NC3=O)C)O]
Added dtdp-3,4-didehydro-2,6-dideoxy-alpha-d-glucose [CC1C(=O)C(=O)CC(O1)OP(=O)([O-])OP(=O)([O-])OCC2C(CC(O2)N3C=C(C(=O)NC3=O)C)O]
Added cystathione [C(CSCC(C(=O)O)N)C(C(=O)O)N]
Added n-acetylglucosamine 1-phosphate [CC(=O)NC1C(C(C(OC1OP(=O)(O)O)CO)O)O]
Added n-acetylglucosamine 1-phosphate [CC(=O)NC1C(C(C(OC1OP(=O)(O)O)CO)O)O]
Added cannabigerolate [CCCCCC1=CC(=C(C(=C1C(=O)O)[O-])CC=C(C)CCC=C(C)C)O]
Added 2-o-(4-deoxy-beta-l-threo-hex-4-enopyranuronosyl)-alpha-l-rhamnopyranose [CC1C(C(C(C(O1)O)OC2C(C(C=C(O2)C(=O)O)O)O)O)O]
Added 2-o-(4-deoxy-beta-l-threo-hex-4-enopyranuron

Added maize starch [OP(=O)(O)[O-].OP(=O)(O)[O-].OP(=O)([O-])[O-].[Ca+2].[Ca+2]]
Added fructose 1,6-bisphosphate [C(C1C(C(C(O1)(COP(=O)(O)O)O)O)O)OP(=O)(O)O]
Added beta,chi-carotene [CC1=C(C(CCC1)(C)C)C=CC(=CC=CC(=CC=CC=C(C)C=CC=C(C)C=CC2=C(C(=C(C=C2)C)C)C)C)C]
Added actinomycin [CC1C(C(=O)NC(C(=O)N2CCCC2C(=O)N(CC(=O)N(C(C(=O)O1)C(C)C)C)C)C(C)C)NC(=O)C3=C4C(=C(C=C3)C)OC5=C(C(=O)C(=C(C5=N4)C(=O)NC6C(OC(=O)C(N(C(=O)CN(C(=O)C7CCCN7C(=O)C(NC6=O)C(C)C)C)C)C(C)C)C)N)C]
Added monovinyl protochlorophyllide a [CCC1=C(C2=NC1=CC3=C(C4=C([N-]3)C(=C5C(=C(C(=N5)C=C6C(=C(C(=C2)[N-]6)C=C)C)C)CCC(=O)[O-])C(=C4[O-])C(=O)OC)C)C.[Mg+2]]
Added orexin b [CCC(C)C(C(=O)NCC(=O)NC(C)C(=O)NC(C)C(=O)NC(CC1=CNC=N1)C(=O)NC(CC(=O)N)C(=O)NCC(=O)NC(CO)C(=O)NC(C)C(=O)NC(CCC(=O)N)C(=O)NC(CC(C)C)C(=O)NC(CC(C)C)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCC(=O)N)C(=O)NC(CC(C)C)C(=O)NC(CCCN=C(N)N)C(=O)NCC(=O)NC(CCC(=O)N)C(=O)NC(CC(C)C)C(=O)NCC(=O)N2CCCC2C(=O)N3CCCC3C(=O)NCC(=O)NC(CO)C(=O)NC(CCCN=C(N)N)C(=O)N)NC(=O)C(CC(C)C)NC(=O)C(C(C)O)NC

In [ ]:
print(Nones2[0]['rifampicin'])